In [2]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import random as rand
import queue
import csv
from collections import OrderedDict
from IPython.display import clear_output
import csv
import re
from heapq import merge
from sklearn import preprocessing
from operator import itemgetter
import bisect
import pickle

from sklearn.datasets import make_circles, make_moons
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from IPython.core.debugger import set_trace
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable
from IPython.display import display
from os import listdir
from os.path import isfile, join

In [57]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        #self.conv1 = nn.Conv2d(1, 64, 7)
        self.linear1_1 = nn.Linear(specsize, 1024)
        self.linear1_2 = nn.Linear(1024, 512)
        self.linear1_3 = nn.Linear(1024, 512)
        #self.linear1_3 = nn.Linear(256, 128)
        self.linear1_4 = nn.Linear(512, 256)
        
        self.lstm2_1 = nn.Linear(specsize, 1024)
        self.linear2_1 = nn.Linear(1024, 512)
        self.linear2_2 = nn.Linear(512, 256)
        #self.pool1 = nn.MaxPool2d(2)
        #self.conv2 = nn.Conv2d(64, 128, 5)
        #self.conv3 = nn.Conv2d(128, 256, 5)
        
        self.linear1 = nn.Linear(128, 2)
        self.linear2 = nn.Linear(64, 2)
        
        self.dropout = nn.Dropout(0.1)
        
        #self.linear2 = nn.Linear(512, 2)
        
    def forward(self, data):
        res = []
        for i in range(data.shape[1]):
            x = data[:, i]
            x = self.linear1_1(x.view(-1, specsize))
            x = F.relu(x)
            x = F.dropout(x, training=self.training)
            x = self.linear1_2(x)
            x = F.relu(x)
            #x = self.linear1_3(x)
            #x = F.relu(x)
            res.append(x)
        
#         x1 = data[:,0]
#         x1 = self.linear1_1(x1.view(-1, specsize))
#         x1 = F.relu(x1)
#         x1 = F.dropout(x1, training=self.training)
        
#         x1 = self.linear1_2(x1)
#         x1 = F.relu(x1)
        
#         x1 = self.linear1_3(x1)
#         x1 = F.relu(x1)
        
# #         x1 = self.linear1_3(x1)
# #         x1 = F.relu(x1)
        
# #         x1 = self.linear1_4(x1)
# #         x1 = F.relu(x1)
        
#         x2 = data[:,1]
        
#         x2 = self.linear1_1(x2.view(-1, specsize))
#         x2 = F.relu(x2)
#         x2 = F.dropout(x2, training=self.training)
        
#         x2 = self.linear1_2(x2)
#         x2 = F.relu(x2)
        
#         x2 = self.linear1_3(x2)
#         x2 = F.relu(x2)
        
#         x2 = self.linear1_3(x2)
#         x2 = F.relu(x2)
        
#         x2 = self.linear1_4(x2)
#         x2 = F.relu(x2)
        #res = F.pairwise_distance(x1, x2)
        #res = torch.abs(x1 - x2)
        #print(res.shape)
        #res = self.linear1(res)
        #res = self.linear2(res)
        
        #return F.log_softmax(res, 1)
        return res
        
#        for i in range(2): # Siamese nets; sharing weights
#            x = data[i]
#            x = self.conv1(x)
#            x = F.relu(x)
#            x = self.pool1(x)
#            x = self.conv2(x)
#            x = F.relu(x)
#            x = self.conv3(x)
#            x = F.relu(x)
#            
#            x = x.view(x.shape[0], -1)
#            x = self.linear1(x)
#            res.append(F.relu(x))
#        
#        res = torch.abs(res[1] - res[0])
#        res = self.linear2(res)
#        return res
    def name(self):
        return "Net"

In [58]:
model = torch.load('models/siamese99.4%working_1.pt')
model.eval()
model

Net(
  (linear1_1): Linear(in_features=8000, out_features=1024, bias=True)
  (linear1_2): Linear(in_features=1024, out_features=512, bias=True)
  (linear1_3): Linear(in_features=1024, out_features=512, bias=True)
  (linear1_4): Linear(in_features=512, out_features=256, bias=True)
  (lstm2_1): Linear(in_features=8000, out_features=1024, bias=True)
  (linear2_1): Linear(in_features=1024, out_features=512, bias=True)
  (linear2_2): Linear(in_features=512, out_features=256, bias=True)
  (linear1): Linear(in_features=128, out_features=2, bias=True)
  (linear2): Linear(in_features=64, out_features=2, bias=True)
  (dropout): Dropout(p=0.1)
)

In [39]:
def runModel(loader):
    with torch.no_grad():
        accurate_labels = 0
        all_labels = 0
        loss = 0
        out = []
        for batch_idx, data in enumerate(loader):

            data = data.to(device)    
            out.extend(model(data)[0].cpu().detach().numpy())
    print(len(out))
    return out

In [40]:
rand.seed(37)
AAs = 'ACDEFGHIKLMNPQRSTVWY'
AAMass = {'A':71.037114, 'C':103.009185, 'D':115.026943, 'E':129.042593, 'F':147.068414, 'G':57.021464, 'H':137.058912,
          'I':113.084064, 'K':128.094963, 'L':113.084064, 'M':131.040485, 'N':114.042927, 'P':97.052764, 'Q':128.058578,
          'R':156.101111, 'S':87.032028, 'T':101.047679, 'V':99.068414, 'W':186.079313, 'Y':163.0633}

H2O = 18.01528
PROTON = 1.00727647
specsize = 8000
precursor_tolerance = 3.0
num_psm = 1
charge = 2
use_mods = False

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 1024

def GetAAMass(AA):
    return AAMass[AA] + 57.021464 if AA == 'C' else AAMass[AA]

def GetSpectrum(Seq):
    Seq = re.sub('[^ACDEFGHIKLMNPQRSTVWY]', '', Seq) #remove non-amino acids
    size = len(Seq)
    outsize = 2*size
    bspectrum = []
    yspectrum = []
    
    bspectrum.append(GetAAMass(Seq[0]) + PROTON)
    yspectrum.append(GetAAMass(Seq[-1]) + H2O + PROTON)
    
    for i, (fAA, bAA) in enumerate(zip((Seq[1:]), Seq[-2::-1])):
        bspectrum.append(bspectrum[i] + GetAAMass(fAA))
        yspectrum.append(yspectrum[i] + GetAAMass(bAA))
        
    mergedout = list(merge(bspectrum, yspectrum))
    if mergedout[-1] > specsize:
        print(mergedout[-1])
        print(Seq)
    tspec = np.zeros(specsize)
    tspec[np.rint(mergedout).astype(int)] = 1
    return tspec

In [42]:
def readmgfs(folderPath, charge=None):
    
    mgffiles = [f for f in listdir(folderPath) if isfile(join(folderPath, f)) and f.split('.')[-1] == 'mgf']
    assert len(mgffiles) > 0
    
    spectra = []
    masses = []
    charges = []
    for file in mgffiles:
        f = open(join(folderPath, file))
        speclines = f.readlines()
        f.close()
        
        if not speclines:
            continue
            
        spec = np.zeros(specsize)
        isMass = isCharge = isSpec = False
        i = 0
        '''Read Headers'''
        while True:
            line = speclines[i]
            i += 1
            
            splits = line.split('=')
            if splits[0].upper() == 'PEPMASS':
                masses.append(float(splits[1].split(' ')[0]))
                isMass = True
                
            if isMass and splits[0].upper() == 'CHARGE':
                l_charge = int(splits[1][0])
                if charge and l_charge != charge:
                    del masses[-1]
                    isMass = False
                    isCharge = False
                else:
                    charges.append(l_charge)
                    isCharge = True
                break
        
        '''Read Spectrum'''
        while isMass and isCharge and i < len(speclines):
            line = speclines[i]
            i += 1
            
            if line != '\n' and 'END IONS' not in line.upper():
                splits = line.split(' ')
                moz, intensity = float(splits[0]), float(splits[1])
                spec[round(moz)] += round(intensity)
            elif 'END IONS' in line.upper():
                break
                
        if isMass and isCharge:        
            spec = np.clip(spec, None, 1000.0)
            spec = preprocessing.scale(spec)
            spectra.append(spec)
        
    return spectra, masses, charges

In [43]:
def readms2(file, charge=None):
    f = open(file)
    lines = f.readlines()
    f.close()
    
    spectra = []
    masses = []
    charges = []
    i = 0
    while i < len(lines):
        line = lines[i][:-1]
        i += 1
        
        splits = line.split('\t')
        if splits[0] == 'Z' and (charge is None or float(splits[1]) == charge):
            charges.append(float(splits[1]))
            masses.append(float(splits[2]))
            spec = np.zeros(specsize)
            while i < len(lines):
                line = lines[i][:-1] #remove the \n character
                i += 1
                splits = line.split(' ')
                if 'S' in splits[0]:
                    break
                if 'Z' in splits[0]:
                    continue
                moz, intensity = float(splits[0]), float(splits[1])
                spec[round(moz)] += round(intensity)
            
            spec = np.clip(spec, None, 1000.0)
            spec = preprocessing.scale(spec)
            spectra.append(spec)
            
    return spectra, masses, charges

In [44]:
ms2file = '/disk/raptor/lclhome/mtari008/Proteomics/crux/specs/demo.ms2'

#queryspectra, spectramasses, _ = readmgfs(specsPath, charge=2)
queryspectra, spectramasses, _ = readms2(ms2file, None)
with torch.no_grad():
    queryspectraTensor = torch.tensor(queryspectra, dtype=torch.float)[:, None, :]
    queryspectra_loader = torch.utils.data.DataLoader(dataset=queryspectraTensor, batch_size=batch_size, shuffle=False)

    queryspectra_out = runModel(queryspectra_loader)
print(len(queryspectra_out))
del queryspectraTensor
del queryspectra_loader
del queryspectra

150
150


In [59]:
def fastatospectra(lines, start, count, dh):
    tspectra = []
    masses = []
    peps = []
    
    new = prev = 0
    end = min(start+count, len(lines))
    for i, line in enumerate(lines[start:end]):
        splits = line.split('\t')
        
        pep = splits[0]
        #print(pep)
        peps.append(pep)
        spec = GetSpectrum(pep)
        tspectra.append(preprocessing.scale(spec))
        masses.append(float(splits[1]))
        
        #print(splits[1])
        '''Progress Monitor'''
        new = int(((i+start)/len(lines)) * 100)
        if (new > prev):
            dh.update(str(new)+'%')
            prev = new
            
    return tspectra, masses, peps

In [61]:
def fastaToSpectraBatch(filePath, spectra_batch_size):    
    f = open(filePath)
    lines = f.readlines()
    f.close()
    masses = []
    spectra_out = []
    peps = []
    
    dh = display('0%', display_id=True)
    
    start = 0
    i = 0
    while start < len(lines):
        print('Batch: ' + str(i))
        i += 1
        
        print('Generating spectra...')
        spectra, l_masses, l_peps = fastatospectra(lines, start, spectra_batch_size, dh)
        masses.extend(l_masses)
        peps.extend(l_peps)
        start = start + spectra_batch_size
        
        with torch.no_grad():
            print('Converting to tensor...')
            '''dtype=torch.float'''
            spectra = np.asarray(spectra)
            spectraTensor = torch.as_tensor(spectra, dtype=torch.float)[:, None, :]
            spectra_loader = torch.utils.data.DataLoader(dataset=spectraTensor, batch_size=batch_size, shuffle=False)
            
            print('Running the model...')
            spectra_out.extend(runModel(spectra_loader))
            
    return spectra_out, masses, peps

In [62]:
spectra_batch_size = 500000
#targetPath = '/disk/raptor/lclhome/mtari008/Proteogenomics/DeepSNAP/data/rat/peptide-dbs/rat-peptides.target.txt'
#decoyPath = '/disk/raptor/lclhome/mtari008/Proteogenomics/DeepSNAP/data/rat/peptide-dbs/rat-peptides.decoy.txt'
targetPath = '/disk/raptor/lclhome/mtari008/Proteomics/crux/crux-output/generate-peptides.target.txt'
decoyPath = '/disk/raptor/lclhome/mtari008/Proteomics/crux/crux-output/generate-peptides.decoy.txt'
targetspectra_out, targetmasses, targetpeptides = fastaToSpectraBatch(targetPath, spectra_batch_size)
decoyspectra_out, decoymasses, decoypeptides = fastaToSpectraBatch(decoyPath, spectra_batch_size)


print(len(targetspectra_out), len(targetmasses))
print(len(decoyspectra_out), len(decoymasses))

'99%'

Batch: 0
Generating spectra...
Converting to tensor...
Running the model...
6005


'99%'

Batch: 0
Generating spectra...
Converting to tensor...
Running the model...
6005
6005 6005
6005 6005


In [66]:
print(targetmasses[0:10])
print(len(decoypeptides))
targetspectra_out, targetmasses, targetpeptides = list(zip(*sorted(zip(targetspectra_out, targetmasses, targetpeptides), 
                                                                   key=lambda pair: pair[1])))
decoyspectra_out, decoymasses, decoypeptides = list(zip(*sorted(zip(decoyspectra_out, decoymasses, decoypeptides),
                                                                key=lambda pair: pair[1])))

(616.3776, 630.3821, 646.3519, 647.3107, 658.3518, 658.4246, 661.3879, 673.3879, 673.3992, 686.4195)
6005


In [50]:
print(len(targetmasses))
print(targetmasses[1000:1010])
print(decoymasses[1000:1010])

6005
(1032.5069, 1032.5319, 1032.5546, 1032.562, 1032.5659, 1032.5871, 1032.6048, 1033.5096, 1033.5426, 1033.546)
(1032.5069, 1032.5319, 1032.5546, 1032.562, 1032.5659, 1032.5871, 1032.6048, 1033.5096, 1033.5426, 1033.546)


In [67]:
def find_range(arr, val, tolerance=3):
    assert len(arr) > 0
    assert tolerance > 0
    assert val > 0
    
    left = val - tolerance if val - tolerance >= 0 else 0
    right = val + tolerance
    
    ileft = bisect.bisect_left(arr, left)
    iright = bisect.bisect_right(arr, right)
    
    return (ileft, iright-1) if iright - ileft > 0 else (-1, -1)
    
# Function to insert element 
def insert(lst, n): 
    assert not lst or len(lst[0]) == len(n)
    
    index = num_psm
    # Searching for the position 
    for i in range(len(lst)): 
        if lst[i][0] > n[0]: 
            index = i 
            break
      
    # Inserting n in the list 
    lst = lst[:index] + [n] + lst[index:] 
    return lst[:num_psm]

In [68]:
def calculate_L2_Dist(queryspectra, querymasses, dbspectra, dbmasses, pre_tol=3, keep_psm=1):
    prev = 0
    psm_scores = []
    psm_pepids = []
    
    for ind, (spec, spec_mass) in enumerate(zip(queryspectra, querymasses)):
        left, right = find_range(dbmasses, spec_mass, pre_tol)
        if left != -1 and right != -1:
            candidate_specs = dbspectra[left:right+1]
            l_scores = np.linalg.norm(np.subtract(candidate_specs, spec), axis=1)**2
            psm_pepids.append(left + l_scores.argsort()[:keep_psm]) #add argsort later
            psm_scores.append(np.sort(l_scores)[:keep_psm])
            #print(ind)
            new = int((ind/len(queryspectra)) * 100)
            if new > prev:
                clear_output(wait=True)
                print(str(new) + '%')
                prev = new
        else:
            print('***********error**************')
            print('ind: ' + str(ind))
            print('spec_mass: ' + str(spec_mass))
            print('**********end error************')
            
    return psm_scores, psm_pepids

In [87]:
def calculate_xcorr(queryspectra, querymasses, dbspectra, dbmasses, pre_tol=3, keep_psm=1):
    prev = 0
    psm_scores = []
    psm_pepids = []
    
    for ind, (spec, spec_mass) in enumerate(zip(queryspectra, querymasses)):
        left, right = find_range(dbmasses, spec_mass, pre_tol)
        if left != -1 and right != -1:
            candidate_specs = dbspectra[left:right+1]
            l_scores = np.dot(candidate_specs, spec)
            psm_pepids.append(left + l_scores.argsort()[::-1][:keep_psm]) #add argsort later
            psm_scores.append(np.sort(l_scores)[::-1][:keep_psm])
            #print(ind)
            new = int((ind/len(queryspectra)) * 100)
            if new > prev:
                clear_output(wait=True)
                print(str(new) + '%')
                prev = new
        else:
            print('***********error**************')
            print('ind: ' + str(ind))
            print('spec_mass: ' + str(spec_mass))
            print('**********end error************')
            
    return psm_scores, psm_pepids

In [88]:
# targetpsm_scores, targetpsm_ids = calculate_L2_Dist(queryspectra_out, spectramasses, targetspectra_out,
#                                      targetmasses, precursor_tolerance, num_psm)
# decoypsm_scores, decoypsm_ids = calculate_L2_Dist(queryspectra_out, spectramasses, decoyspectra_out,
#                                      decoymasses, precursor_tolerance, num_psm)
targetpsm_scores, targetpsm_ids = calculate_xcorr(queryspectra_out, spectramasses, targetspectra_out,
                                     targetmasses, precursor_tolerance, num_psm)
decoypsm_scores, decoypsm_ids = calculate_xcorr(queryspectra_out, spectramasses, decoyspectra_out,
                                     decoymasses, precursor_tolerance, num_psm)

99%


In [54]:
print(targetmasses[2000000:2000010])

()


In [89]:
print(len(targetspectra_out))
print(len(decoyspectra_out))

print(len(targetpsm_scores))
print(len(decoypsm_scores))

print(targetpsm_scores[0:10])
print(decoypsm_scores[0:10])

targetpsm_scores = np.asarray(targetpsm_scores).flatten()
decoypsm_scores = np.asarray(decoypsm_scores).flatten()

targetpsm_ids = np.asarray(targetpsm_ids).flatten()
decoypsm_ids = np.asarray(decoypsm_ids).flatten()

targetpsm_peps = np.asarray(targetpeptides)[targetpsm_ids]
decoypsm_peps = np.asarray(decoypeptides)[decoypsm_ids]

#targetpsm_ids = targetpsm_ids[np.argsort(np.asarray(targetpsm_scores).flatten())]
#decoypsm_ids = decoypsm_ids[np.argsort(np.asarray(decoypsm_scores).flatten())]

sorted_targets = np.sort(np.asarray(targetpsm_scores).flatten())[::-1]
sorted_decoys = np.sort(np.asarray(decoypsm_scores).flatten())[::-1]

print(targetpsm_ids[0:10])
print(decoypsm_ids[0:10])

print(sorted_targets[0:100])
print(sorted_decoys[0:100])

#targetpsm_scores = np.column_stack((targetpsm_scores, np.ones(len(targetpsm_scores))))
#decoypsm_scores = np.column_stack((decoypsm_scores, np.zeros(len(decoypsm_scores))))
#decoypsm_scores = np.asarray(decoypsm_scores)

6005
6005
150
150
[array([119.93341], dtype=float32), array([119.42667], dtype=float32), array([92.465195], dtype=float32), array([70.95618], dtype=float32), array([81.353226], dtype=float32), array([98.86684], dtype=float32), array([86.25241], dtype=float32), array([101.92701], dtype=float32), array([88.03976], dtype=float32), array([87.60692], dtype=float32)]
[array([120.65355], dtype=float32), array([131.83395], dtype=float32), array([83.25646], dtype=float32), array([75.8531], dtype=float32), array([75.34193], dtype=float32), array([103.34235], dtype=float32), array([101.130066], dtype=float32), array([95.64265], dtype=float32), array([91.43796], dtype=float32), array([81.032326], dtype=float32)]
[1825 2519 2290   32 1299 2443   28  842 5679 1621]
[1829 2519 2297   29 1304 2443   27  839 5680 1623]
[195.31476  155.87149  154.71373  153.74385  144.41605  140.9835
 139.00238  138.27695  137.98456  136.98334  136.33673  134.513
 132.89551  129.94763  126.72832  125.20219  123.55231  1

In [90]:
targetpsm_scores

array([119.93341 , 119.42667 ,  92.465195,  70.95618 ,  81.353226,
        98.86684 ,  86.25241 , 101.92701 ,  88.03976 ,  87.60692 ,
        92.62938 ,  88.97537 ,  97.592606,  85.64073 ,  67.35411 ,
       138.27695 , 154.71373 , 139.00238 ,  94.75311 ,  82.6724  ,
        87.70526 ,  90.36142 ,  81.642944,  89.02522 , 101.45397 ,
        71.009445,  78.95327 ,  85.16355 ,  80.693184,  86.22511 ,
       105.39874 , 117.95119 ,  83.0087  ,  83.967026, 113.95644 ,
        65.299255,  83.3947  ,  94.426346, 111.89251 , 117.721634,
       121.66821 ,  80.59951 ,  80.03313 , 134.513   ,  72.0045  ,
        73.785484,  79.995514, 113.875946, 103.37543 ,  82.021805,
        67.84793 , 105.61763 , 125.20219 ,  84.75174 ,  67.29022 ,
        71.47301 ,  72.40285 , 155.87149 ,  76.36668 , 111.81023 ,
       126.72832 ,  90.68437 , 117.92219 ,  77.77935 ,  75.91037 ,
       144.41605 ,  80.47797 , 129.94763 ,  95.9413  ,  85.37846 ,
        93.407906,  79.620384,  75.57164 ,  76.36846 ,  72.402

In [91]:
df = pd.DataFrame(
    data={'spectrum_ids':np.arange(len(targetpsm_peps)), 'target_psms':targetpsm_peps, 
          'targetpsm_scores':targetpsm_scores, 'decoy_psms':decoypsm_peps, 'decoypsm_scores':decoypsm_scores})
df.to_csv('deepsearch.csv')

In [ ]:
sortedpeps = np.array(sortedpeps)
correct_matches = 0
for ind, psms in enumerate(psm_scores):
    #print('ind: ' + str(ind) + ', mass: ' + str(sortedspecmasses[ind]))
    #print('psms: ' + str(psms) + ', psm mass: ' + str(sortedpepmasses[psms[0]]))
    cand_peps = sortedpeps[psms]
    #print(cand_peps)
    for pep in cand_peps:
        #print('pepidmass: ' + str(pepidmass[pep][0]))
        #print('ids: ' + str(ids[ind]) + ', in: ' + str(pepidmass[pep][0]))
        if ids[ind] in pepidmass[pep][0]:
            correct_matches += 1
            break
print(len(psm_scores))
print(correct_matches)
print(str((correct_matches/len(psm_scores)) * 100) + '% correct matches in top ' + str(num_psm) + ' psms.')

In [ ]:
print(5184 - 1739)

In [ ]:
file = open('/disk/raptor/lclhome/mtari008/Proteogenomics/DeepSNAP/data/rat/peptide-dbs/rat-peptides.target.txt')
tlines = file.readlines()
print(len(tlines))
tpeps = list((tline.split('\t')[0], float(tline.split('\t')[1])) for tline in tlines)
tpeps = sorted(tpeps, key=lambda pair: pair[1])
print(tpeps[10000:10010])
file.close()

In [ ]:
file = open('/disk/raptor/lclhome/mtari008/Proteogenomics/DeepSNAP/data/rat/peptide-dbs/rat-peptides.decoy.txt')
tlines = file.readlines()
print(len(tlines))
tpeps = list((tline.split('\t')[0], float(tline.split('\t')[1])) for tline in tlines)
tpeps = sorted(tpeps, key=lambda pair: pair[1])
print(tpeps[10000:10010])
file.close()

In [72]:
np.arange(150)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149])